Placeholder

In [37]:
import random
import numpy as np
import math
import itertools


np.random.seed(21098573)
class toy_example_c():
  #k, max iterations, whereas here, we will be using the stopping criteria...
  #mu, convexity constants
  #M - Matrix
  #d - Dimension
  #a position of saddle point in \X
  #b position of saddle point in \Y

  def __init__(self):

    self.mu = 0
    self.d = 4
    self.z_0 = [np.ones(self.d), np.zeros(self.d)];
    self.M = np.array([[random.uniform(-0.1, 0.1) for _ in range(self.d)] for _ in range(self.d)])
    self.X = np.array([random.uniform(0, 1) for _ in range(self.d)])
    self.Y = np.array([random.uniform(0, 1) for _ in range(self.d)])
    #x* and y*
    self.a = np.random.uniform(0.25, 0.75, self.d)#position of saddle point in x
    self.b = np.random.uniform(0.25, 0.75, self.d)#position of saddle point in y #preset solution?

    self.p_width = 1/math.sqrt(self.d) #defined as some very long equation about cones and faces
    #self.delta = min(euclidian_norm(min([1-self.a, self.a])), euclidian_norm(min([1-self.b,self.b])));
    self.diam = math.sqrt(self.d)
    self.C = self.mu * math.pow(self.diam,2)
    self.tolerance = 1e-6
    #diameter is given but defined as sup min s ||x*-s||


  def function(self,x,y):
    x, y, a, b = np.array(x), np.array(y), np.array(self.a), np.array(self.b)
    term1 = (self.mu/2) * math.pow(euclidian_norm(x - a), 2)
    #this is mu/2 * ||x - x*||^2
    term2 =  np.dot(x - a, np.dot(self.M, y - b))
    #this is (x-x*).TM(y-y*)
    term3 = (self.mu/2)*math.pow(euclidian_norm(y - b), 2)
    #this is mu/2 * ||y - y*||^2
    return term1 + term2 - term3

  def df_dx(self, x, y):
    term1_grad = self.mu * (x - self.a)
    term2_grad = np.dot(self.M, (y - self.b))
    return term1_grad + term2_grad

  def df_dy(self, x,y):
    term2_grad = np.dot(self.M.T, (x - self.a))
    term3_grad = -self.mu * (y - self.b)
    return term2_grad + term3_grad

  def stopping_criteria(self,x,y):
    #true when we should STOP

    return (self.gFW(x,y) < self.tolerance)

  def gFW(self, x_t,y_t):

    grad_x_L = self.df_dx(x_t, y_t)
    grad_y_L = self.df_dy(x_t, y_t)

    # Maximize over set X
    max_val_x = -np.inf  # Start with a very small number
    for sx in self.X:
        current_val = np.dot(x_t - sx, grad_x_L)  # Compute the inner product
        if current_val > max_val_x:
            max_val_x = current_val  # Update the maximum value if current is larger

    # Maximize over set Y
    max_val_y = -np.inf  # Start with a very small number
    for sy in self.Y:
        current_val = np.dot(y_t - sy, -grad_y_L)  # Compute the inner product
        if current_val > max_val_y:
            max_val_y = current_val  # Update the maximum value if current is larger

    return max_val_x + max_val_y

  def LMO(self, x,y):
    return [self.df_dx(x,y), self.df_dy(x,y)]

  def r(self, x,y):
    u = self.df_dx(x,y)
    #print(u)
    v = (-1)*self.df_dy(x,y)
    #print(v)
    return np.concatenate((u,v))

  def s(self,x,y):
    # Adding a small epsilon to handle numerical stability near zero
    eps = np.finfo(float).eps
    return (-1)*((np.sign(function(x,y) +eps) - 1)*.5)

  def g(self,x,y):
    return np.dot(self.s(x,y))

  def my_s(self,x,y):
    XxY = itertools.product(self.X, self.Y)
    XxY = list(XxY)
    print(XxY)
    rt = self.r(x,y)
    rt = np.array(rt[0] + rt[1])

    min_value = [np.inf, np.inf]
    z_min = None
    for z in XxY:
      z = np.array(z)
      #print("z shape:", z.shape)
      #print(rt)
      c_val = np.dot(z,rt)
      #print("c_val", c_val)
      if(all([c_val[i] < min_value[i] for i in range(0, len(c_val))])):
        min_value = c_val
        z_min = z

    return z_min


def border_distance(x):
  return np.argmin()

def euclidian_norm(x):
  #I just wrote this method to make it explicit what the norm is that I am using
  #and to not rely too much on using outside libraries where I can
  tot = 0
  for i in range(0, len(x)):
    tot += math.pow(x[i],2)
  return math.pow(tot, 0.5)



max_iters = 500
alpha = 1
G = []

toy = toy_example_c()

def SP_FW(f, max_iters, step_size_algo):
  x = np.array(f.z_0[0])
  y = np.array(f.z_0[1])
  for t in range(0, max_iters):
    rt = f.r(x,y)
    st = f.my_s(x,y)
    st = np.array(st)

    gt = np.dot(np.array([x,y]) - st,rt)
    if(gt < f.tolerance):
      return [x,y]
    else:
      if(step_size_algo == "fw"):
        step_size = 2/(2+t)
      z = (1-step_size)*np.hstack([x,y]) + step_size*st
      x = z[0]
      y = z[1]
      #just to update iterates
  return [x, y]


print(SP_FW(toy,max_iters,"fw"))


[(0.6671178723451998, 0.15395477831729087), (0.6671178723451998, 0.08673173591040662), (0.6671178723451998, 0.9568117182809636), (0.6671178723451998, 0.8741143125236136), (0.5958689594735826, 0.15395477831729087), (0.5958689594735826, 0.08673173591040662), (0.5958689594735826, 0.9568117182809636), (0.5958689594735826, 0.8741143125236136), (0.29449083657688746, 0.15395477831729087), (0.29449083657688746, 0.08673173591040662), (0.29449083657688746, 0.9568117182809636), (0.29449083657688746, 0.8741143125236136), (0.6524808973234792, 0.15395477831729087), (0.6524808973234792, 0.08673173591040662), (0.6524808973234792, 0.9568117182809636), (0.6524808973234792, 0.8741143125236136)]


ValueError: operands could not be broadcast together with shapes (2,4) (2,) 